In [1]:
import numpy as np
# import seq2seq
# from seq2seq.models import SimpleSeq2Seq
import keras.backend as K
import gensim
import keras
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense, GRU, Embedding, Bidirectional, BatchNormalization, TimeDistributed
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.utils import to_categorical
from keras.callbacks import History ,ModelCheckpoint, EarlyStopping
from keras.layers.merge import add, dot, concatenate
import numpy as np
import random
import pandas as pd
%env CUDA_VISIBLE_DEVICES=7
import pickle

Using TensorFlow backend.


env: CUDA_VISIBLE_DEVICES=7


In [2]:
train_data = np.load("data/train.data")
test_data = np.load("data/test.data")
print(len(train_data),"個 training 音檔")
print(len(test_data),"個 testing 音檔")
max_frame_length = np.max([len(sample) for sample in train_data])
print("max langth of wav:",max_frame_length)

45036 個 training 音檔
2000 個 testing 音檔
max langth of wav: 246


In [3]:
# load caption
with open("data/train.caption","r") as f:
    train_caption = f.readlines()
    train_caption = [sent.strip() for sent in train_caption]
    train_sentences = [sent.split(" ") for sent in train_caption]
with open("data/test.csv","r") as f:
    test_choice = f.readlines()
    test_choice = [sent.strip() for sent in test_choice]
    test_corpus = ",".join(test_choice)
    test_sentences = [sent.split(" ") for sent in test_corpus.split(",")]
    test_corpus = test_corpus.replace(",", " ")

In [4]:
# chinese character level tokenizer
tokenizer = Tokenizer(num_words=None,filters='\n', lower=True, split=" ", char_level=False)
tokenizer.fit_on_texts(train_caption + [test_corpus])
print("number of token in caption:", len(tokenizer.word_index))
inv_map = {v: k for k, v in tokenizer.word_index.items()}

number of token in caption: 2446


In [5]:
train_caption_sequences = tokenizer.texts_to_sequences(train_caption)
max_length = np.max([len(i) for i in train_caption_sequences])
print("max length:", max_length)

max length: 13


In [6]:
# pad sequence
train_caption_pad = pad_sequences(train_caption_sequences, maxlen=max_length)
train_data_pad = pad_sequences(train_data, maxlen=max_frame_length,dtype='float32')
# revert
print([inv_map[i] for i in  train_caption_pad[1] if i != 0])
print(train_caption_pad.shape)
print(train_data_pad.shape)

['對', '了', '文', '隆', '哥']
(45036, 13)
(45036, 246, 39)


In [7]:
# # build embedding matrix
# embedding_matrix = np.zeros((len(tokenizer.word_index)+1, emb_size))
# for word, i in tokenizer.word_index.items():
#     try:
#         embedding_vector = w2v_model.wv[word]
#         embedding_matrix[i] = embedding_vector
#     except:
#         oov_count +=1
#         print(word)
# print("embedding matrix shape:",embedding_matrix.shape)

In [8]:
# build training tensor (truth and fake for binary calssification)
sent_word_count = np.count_nonzero(train_caption_pad,axis=1)
false_caption = []
false_mfcc = train_data_pad
true_caption = train_caption_pad
true_mfcc = train_data_pad

# sample fake caption with same length
# for i,sent in enumerate(train_caption_pad):
#     current_word_count = sent_word_count[i]
#     fake_sent = train_caption_pad[np.random.choice(np.where(sent_word_count==current_word_count)[0],1)]
#     if list(fake_sent) != list(sent):
#         false_caption += list(fake_sent)
#     else:
#         fake_sent = train_caption_pad[np.random.choice(np.where(sent_word_count==current_word_count)[0],1)]
#         false_caption += list(fake_sent)
#         print("replicate sample")

## rolling way
false_caption = np.concatenate((np.roll(train_caption_pad,1,axis=0),
                                np.roll(train_caption_pad,2,axis=0),
                               np.roll(train_caption_pad,3,axis=0),
                               np.roll(train_caption_pad,4,axis=0),
                               np.roll(train_caption_pad,5,axis=0)))
false_mfcc = np.concatenate((train_data_pad,
                             train_data_pad,
                             train_data_pad,
                             train_data_pad,
                             train_data_pad))
true_caption = train_caption_pad
true_mfcc = train_data_pad

In [9]:
# for _ in range(1):
#     false_mfcc += list(train_data_pad)
#     np.random.shuffle(_train_caption_pad)
#     false_caption += list(_train_caption_pad)
ground_truth = [ 1 for _ in range(len(true_caption))] + [0 for _ in range(len(false_caption))]
train_mfcc = np.concatenate((true_mfcc, np.array(false_mfcc)))
train_caption = np.concatenate((true_caption, np.array(false_caption)))

In [10]:
print(train_mfcc.shape)
print(train_caption.shape)

(270216, 246, 39)
(270216, 13)


In [11]:
print(train_caption_pad[4])
print("where",np.where(np.count_nonzero(train_caption_pad,axis=1)==5)[0])
np.random.choice(np.where(np.count_nonzero(train_caption_pad,axis=1)==5)[0],1)

[  0   0   0   0   0   0  48  16   3   2 939 872   1]
where [    1    17    39 ..., 45017 45018 45026]


array([3033])

In [12]:
len(train_caption)

270216

In [ ]:
# model
emb_size = 100
batch_size = 512
epochs = 50
for i in range(0,40):
    mfcc_input = Input(shape=(train_mfcc.shape[1],train_mfcc.shape[2]))
    mfcc_lstm1 = Bidirectional(LSTM(128,dropout=0.2, return_sequences=True))(mfcc_input)
    mfcc_lstm2 = Bidirectional(LSTM(64,dropout=0.2, return_sequences=True))(mfcc_lstm1)
    mfcc_lstm3 = Bidirectional(LSTM(32,dropout=0.2))(mfcc_lstm2)
    
    caption_input = Input(shape=(13,))
    emb = Embedding(len(tokenizer.word_index)+1 ,output_dim= emb_size, 
                    input_length=max_length,trainable=True)(caption_input)
    caption_lstm1 = Bidirectional(LSTM(128,dropout=0.2, return_sequences = True))(emb)
    caption_lstm2 = Bidirectional(LSTM(64,dropout=0.2, return_sequences = True))(caption_lstm1)
    caption_lstm3 = Bidirectional(LSTM(32,dropout=0.2))(caption_lstm2)
    
    merge = keras.layers.dot([mfcc_lstm3, caption_lstm3],1)
    output_dense = Dense(1,activation="sigmoid")(merge)
    model = Model(inputs=[mfcc_input, caption_input], outputs=output_dense)
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['acc'])
    print(model.summary())
    # training
    total_sample_size = len(ground_truth)
    random_index = np.random.choice(total_sample_size,total_sample_size, replace=False)
    
    input_mfcc = train_mfcc[random_index]
    input_caption = train_caption[random_index]
    input_ground_truth = np.array(ground_truth)[random_index]

    hist = History()
#     check_save  = ModelCheckpoint("models/model_1v3-{epoch:05d}-{val_acc:.5f}.h5",monitor='val_acc',save_best_only=True)
    check_save  = ModelCheckpoint("models/model_1V5_3layers_"+str(batch_size)+"_"+str(i)+".h5",monitor='val_acc',save_best_only=True)
    early_stop = EarlyStopping(monitor="val_loss", patience=4)
    model.fit([input_mfcc, input_caption], input_ground_truth,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.05,
             callbacks=[check_save, hist, early_stop])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 13)            0                                            
____________________________________________________________________________________________________
input_1 (InputLayer)             (None, 246, 39)       0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 13, 100)       244700      input_2[0][0]                    
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 246, 256)      172032      input_1[0][0]                    
___________________________________________________________________________________________

Train on 256705 samples, validate on 13511 samples
Epoch 1/50
256705/256705 [==============================] - 1119s - loss: 0.4429 - acc: 0.8322 - val_loss: 0.4134 - val_acc: 0.8318
Epoch 2/50
256705/256705 [==============================] - 1122s - loss: 0.4029 - acc: 0.8334 - val_loss: 0.3909 - val_acc: 0.8332
Epoch 3/50
256705/256705 [==============================] - 1098s - loss: 0.3903 - acc: 0.8339 - val_loss: 0.3913 - val_acc: 0.8327
Epoch 4/50
256705/256705 [==============================] - 1102s - loss: 0.3827 - acc: 0.8340 - val_loss: 0.3821 - val_acc: 0.8360
Epoch 5/50
256705/256705 [==============================] - 1089s - loss: 0.3740 - acc: 0.8351 - val_loss: 0.3712 - val_acc: 0.8356
Epoch 6/50
256705/256705 [==============================] - 1097s - loss: 0.3597 - acc: 0.8380 - val_loss: 0.3568 - val_acc: 0.8390
Epoch 7/50
256705/256705 [==============================] - 1096s - loss: 0.3462 - acc: 0.8416 - val_loss: 0.3438 - val_acc: 0.8428
Epoch 8/50
256705/256705 

256705/256705 [==============================] - 1136s - loss: 0.2752 - acc: 0.8774 - val_loss: 0.3069 - val_acc: 0.8625
Epoch 17/50
256705/256705 [==============================] - 1121s - loss: 0.2672 - acc: 0.8808 - val_loss: 0.2998 - val_acc: 0.8673
Epoch 18/50
256705/256705 [==============================] - 1131s - loss: 0.2642 - acc: 0.8828 - val_loss: 0.3065 - val_acc: 0.8663
Epoch 19/50
256705/256705 [==============================] - 1109s - loss: 0.2568 - acc: 0.8870 - val_loss: 0.3050 - val_acc: 0.8668
Epoch 20/50
256705/256705 [==============================] - 1077s - loss: 0.2503 - acc: 0.8901 - val_loss: 0.2970 - val_acc: 0.8696
Epoch 21/50
256705/256705 [==============================] - 1091s - loss: 0.2438 - acc: 0.8929 - val_loss: 0.2990 - val_acc: 0.8731
Epoch 22/50
256705/256705 [==============================] - 1087s - loss: 0.2375 - acc: 0.8969 - val_loss: 0.2985 - val_acc: 0.8743
Epoch 23/50
256705/256705 [==============================] - 1075s - loss: 0.2323

Train on 256705 samples, validate on 13511 samples
Epoch 1/50
256705/256705 [==============================] - 996s - loss: 0.4410 - acc: 0.8319 - val_loss: 0.4167 - val_acc: 0.8332
Epoch 2/50
256705/256705 [==============================] - 994s - loss: 0.4070 - acc: 0.8333 - val_loss: 0.3950 - val_acc: 0.8328
Epoch 3/50
256705/256705 [==============================] - 1021s - loss: 0.3882 - acc: 0.8341 - val_loss: 0.3867 - val_acc: 0.8312
Epoch 4/50
256705/256705 [==============================] - 986s - loss: 0.3742 - acc: 0.8354 - val_loss: 0.3704 - val_acc: 0.8361
Epoch 5/50
256705/256705 [==============================] - 980s - loss: 0.3609 - acc: 0.8378 - val_loss: 0.3585 - val_acc: 0.8402
Epoch 6/50
256705/256705 [==============================] - 1018s - loss: 0.3456 - acc: 0.8429 - val_loss: 0.3484 - val_acc: 0.8417
Epoch 7/50
256705/256705 [==============================] - 1025s - loss: 0.3346 - acc: 0.8475 - val_loss: 0.3470 - val_acc: 0.8420
Epoch 8/50
256705/256705 [===

256705/256705 [==============================] - 1048s - loss: 0.2750 - acc: 0.8782 - val_loss: 0.3057 - val_acc: 0.8696
Epoch 17/50
256705/256705 [==============================] - 1167s - loss: 0.2663 - acc: 0.8822 - val_loss: 0.3018 - val_acc: 0.8704
Epoch 18/50
256705/256705 [==============================] - 1021s - loss: 0.2605 - acc: 0.8856 - val_loss: 0.3054 - val_acc: 0.8674
Epoch 19/50
256705/256705 [==============================] - 948s - loss: 0.2548 - acc: 0.8885 - val_loss: 0.3127 - val_acc: 0.8679
Epoch 20/50
256705/256705 [==============================] - 946s - loss: 0.2487 - acc: 0.8911 - val_loss: 0.3046 - val_acc: 0.8700
Epoch 21/50
256705/256705 [==============================] - 950s - loss: 0.2428 - acc: 0.8944 - val_loss: 0.3034 - val_acc: 0.8717
Epoch 22/50
256705/256705 [==============================] - 922s - loss: 0.2375 - acc: 0.8967 - val_loss: 0.3056 - val_acc: 0.8715
_____________________________________________________________________________________

Train on 256705 samples, validate on 13511 samples
Epoch 1/50
256705/256705 [==============================] - 1055s - loss: 0.4421 - acc: 0.8317 - val_loss: 0.4272 - val_acc: 0.8335
Epoch 2/50
256705/256705 [==============================] - 951s - loss: 0.4200 - acc: 0.8335 - val_loss: 0.4136 - val_acc: 0.8338
Epoch 3/50
256705/256705 [==============================] - 961s - loss: 0.3991 - acc: 0.8339 - val_loss: 0.3911 - val_acc: 0.8329
Epoch 4/50
256705/256705 [==============================] - 963s - loss: 0.3809 - acc: 0.8349 - val_loss: 0.3760 - val_acc: 0.8372
Epoch 5/50
256705/256705 [==============================] - 961s - loss: 0.3648 - acc: 0.8369 - val_loss: 0.3613 - val_acc: 0.8361
Epoch 6/50
256705/256705 [==============================] - 966s - loss: 0.3491 - acc: 0.8415 - val_loss: 0.3491 - val_acc: 0.8408
Epoch 7/50
256705/256705 [==============================] - 962s - loss: 0.3350 - acc: 0.8470 - val_loss: 0.3378 - val_acc: 0.8445
Epoch 8/50
256705/256705 [=====

256705/256705 [==============================] - 1012s - loss: 0.3126 - acc: 0.8567 - val_loss: 0.3293 - val_acc: 0.8488
Epoch 15/50
256705/256705 [==============================] - 1027s - loss: 0.3062 - acc: 0.8593 - val_loss: 0.3316 - val_acc: 0.8482
Epoch 16/50
256705/256705 [==============================] - 1017s - loss: 0.2993 - acc: 0.8634 - val_loss: 0.3275 - val_acc: 0.8515
Epoch 17/50
256705/256705 [==============================] - 1063s - loss: 0.2943 - acc: 0.8664 - val_loss: 0.3266 - val_acc: 0.8549
Epoch 18/50
 19968/256705 [=>............................] - ETA: 985s - loss: 0.2869 - acc: 0.8703

In [8]:
test_caption_sequences =  tokenizer.texts_to_sequences([" ".join(sample) for sample in test_sentences])

# pad sequence
test_caption_pad = pad_sequences(test_caption_sequences, maxlen=max_length)
test_data_pad = pad_sequences(test_data, maxlen=max_frame_length,dtype='float32')
test_data_pad_expand = np.repeat(test_data_pad, 4,axis=0)
# revert
print(test_caption_pad.shape)
print(test_data_pad_expand .shape)

(8000, 13)
(8000, 246, 39)


In [9]:
p = []
for i in range(10):
    print(i)
    model = load_model("models/model_1V5_512_"+str(i)+"_share1_bn.h5")
    prediction = model.predict([test_data_pad_expand,test_caption_pad])
    p.append(prediction)

0
1
2
3
4
5
6
7
8
9


In [20]:
prediction+prediction

array([[ 0.56048602],
       [ 0.24864547],
       [ 0.40250978],
       ..., 
       [ 0.18335563],
       [ 0.0273765 ],
       [ 0.59960878]], dtype=float32)

In [32]:
# load submit
sample_submit = pd.read_csv("./data/sample_submission.csv")
pred_y = np.argmax(pred_y_prob.reshape(-1,4),axis=1)
sample_submit["answer"] = pred_y
sample_submit.to_csv("final_submission.csv",index=None)

In [52]:
# model with shared weight 
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
set_session(tf.Session(config=config))

emb_size = 100
batch_size = 512
epochs = 50
for i in range(10):
    share_lstm1 = Bidirectional(LSTM(64,dropout=0.2))
    mfcc_input = Input(shape=(train_mfcc.shape[1],train_mfcc.shape[2]))
    mfcc_lstm1 = Bidirectional(LSTM(128,dropout=0.2,return_sequences=True))(mfcc_input)
    mfcc_lstm2 = share_lstm1(mfcc_lstm1)
    mfcc_bn2 = BatchNormalization()(mfcc_lstm2)
    
    caption_input = Input(shape=(13,))
    emb = Embedding(len(tokenizer.word_index)+1 ,output_dim= emb_size, 
                    input_length=max_length,trainable=True)(caption_input)
    caption_lstm1 = Bidirectional(LSTM(128,dropout=0.2,return_sequences = True))(emb)
    caption_lstm2 = share_lstm1(caption_lstm1)
    caption_bn2 = BatchNormalization()(caption_lstm2)
    
    merge = keras.layers.dot([mfcc_bn2, caption_bn2],1)
    output_dense = Dense(1,activation="sigmoid")(merge)
    model = Model(inputs=[mfcc_input, caption_input], outputs=output_dense)
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['acc'])
    print(model.summary())
    # training
    total_sample_size = len(ground_truth)
    random_index = np.random.choice(total_sample_size,total_sample_size, replace=False)
    
    input_mfcc = train_mfcc[random_index]
    input_caption = train_caption[random_index]
    input_ground_truth = np.array(ground_truth)[random_index]

    hist = History()
#     check_save  = ModelCheckpoint("models/model_1v3-{epoch:05d}-{val_acc:.5f}.h5",monitor='val_acc',save_best_only=True)
    check_save  = ModelCheckpoint("models/model_1V5_"+str(batch_size)+"_"+str(i)+"_share1_bn.h5",monitor='val_acc',save_best_only=True)
    early_stop = EarlyStopping(monitor="val_loss", patience=4)
    model.fit([input_mfcc, input_caption], input_ground_truth,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.05,
             callbacks=[check_save, hist, early_stop])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_7 (InputLayer)             (None, 13)            0                                            
____________________________________________________________________________________________________
input_6 (InputLayer)             (None, 246, 39)       0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 13, 100)       244700      input_7[0][0]                    
____________________________________________________________________________________________________
bidirectional_13 (Bidirectional) (None, 246, 256)      172032      input_6[0][0]                    
___________________________________________________________________________________________

Train on 256705 samples, validate on 13511 samples
Epoch 1/50
256705/256705 [==============================] - 992s - loss: 0.5190 - acc: 0.7659 - val_loss: 0.4525 - val_acc: 0.8347
Epoch 2/50
256705/256705 [==============================] - 743s - loss: 0.4362 - acc: 0.8323 - val_loss: 0.6366 - val_acc: 0.8343
Epoch 3/50
256705/256705 [==============================] - 734s - loss: 0.4122 - acc: 0.8326 - val_loss: 0.5153 - val_acc: 0.8295
Epoch 4/50
256705/256705 [==============================] - 745s - loss: 0.3912 - acc: 0.8342 - val_loss: 0.4186 - val_acc: 0.8156
Epoch 5/50
256705/256705 [==============================] - 738s - loss: 0.3703 - acc: 0.8387 - val_loss: 0.3975 - val_acc: 0.8192
Epoch 6/50
256705/256705 [==============================] - 734s - loss: 0.3502 - acc: 0.8449 - val_loss: 0.3688 - val_acc: 0.8359
Epoch 7/50
256705/256705 [==============================] - 740s - loss: 0.3326 - acc: 0.8512 - val_loss: 0.3574 - val_acc: 0.8411
Epoch 8/50
256705/256705 [======

Train on 256705 samples, validate on 13511 samples
Epoch 1/50
256705/256705 [==============================] - 970s - loss: 0.5334 - acc: 0.7643 - val_loss: 0.4652 - val_acc: 0.8307
Epoch 2/50
256705/256705 [==============================] - 678s - loss: 0.4312 - acc: 0.8317 - val_loss: 0.5467 - val_acc: 0.7598
Epoch 3/50
256705/256705 [==============================] - 697s - loss: 0.4165 - acc: 0.8325 - val_loss: 1.1110 - val_acc: 0.5537
Epoch 4/50
256705/256705 [==============================] - 673s - loss: 0.4015 - acc: 0.8339 - val_loss: 0.7519 - val_acc: 0.6484
Epoch 5/50
256705/256705 [==============================] - 715s - loss: 0.3855 - acc: 0.8366 - val_loss: 0.6231 - val_acc: 0.6751
Epoch 6/50
256705/256705 [==============================] - 704s - loss: 0.3676 - acc: 0.8400 - val_loss: 0.4838 - val_acc: 0.7854
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #

256705/256705 [==============================] - 713s - loss: 0.3149 - acc: 0.8592 - val_loss: 0.3536 - val_acc: 0.8405
Epoch 10/50
256705/256705 [==============================] - 657s - loss: 0.3037 - acc: 0.8643 - val_loss: 0.3434 - val_acc: 0.8485
Epoch 11/50
256705/256705 [==============================] - 653s - loss: 0.2938 - acc: 0.8693 - val_loss: 0.3403 - val_acc: 0.8457
Epoch 12/50
256705/256705 [==============================] - 657s - loss: 0.2844 - acc: 0.8738 - val_loss: 0.3367 - val_acc: 0.8514
Epoch 13/50
256705/256705 [==============================] - 656s - loss: 0.2758 - acc: 0.8778 - val_loss: 0.3319 - val_acc: 0.8586
Epoch 14/50
256705/256705 [==============================] - 659s - loss: 0.2661 - acc: 0.8830 - val_loss: 0.3266 - val_acc: 0.8586
Epoch 15/50
256705/256705 [==============================] - 656s - loss: 0.2593 - acc: 0.8862 - val_loss: 0.3242 - val_acc: 0.8608
Epoch 16/50
256705/256705 [==============================] - 665s - loss: 0.2510 - acc: 

____________________________________________________________________________________________________
bidirectional_37 (Bidirectional) (None, 246, 256)      172032      input_22[0][0]                   
____________________________________________________________________________________________________
bidirectional_38 (Bidirectional) (None, 13, 256)       234496      embedding_9[0][0]                
____________________________________________________________________________________________________
bidirectional_36 (Bidirectional) (None, 128)           164352      bidirectional_37[0][0]           
                                                                   bidirectional_38[0][0]           
____________________________________________________________________________________________________
batch_normalization_20 (BatchNor (None, 128)           512         bidirectional_36[0][0]           
___________________________________________________________________________________________